# Домашнее задание № 3. Исправление опечаток

## 1. Учет грамматики при оценке исправлений (3 балла)

В последнюю итерацию алгоритма для генерации исправлений добавьте еще один компонент - учет грамматической информации. Частично она уже учитывается за счет языковой модели (вероятность предсказывается для словоформы), но такой подход ограничен из-за того, что модель не может ничего предсказать для словоформ, которых не было в обучающей выборке. Чтобы это исправить постройте еще одну "языковую модель" на грамматических тэгах:
1) Используя mystem или pymorphy, разметьте какой-нибудь корпус (например, кусок wiki из семинара) или воспользуйтесь уже размеченным корпусом (например, opencorpora)
2) соберите униграмные и биграмные статистики на уровне грамматических тэгов (например, вместо `задача важна` у вас будет биграм `S,жен,неод=им,ед A=ед,кр,жен`). Для простоты можете начать только с частеречных тэгов и добавить остальную информацию позже
3) напишите функцию, которая будет оценивать вероятность данного предложения на основе грамматической языковой модели (статистик из предыдущего шага). Функция должна сначала преобразовать текст в грамматические тэги, используя точно такой же подход, что использовался на шаге 1. 
4) в функции correct_text_with_lm замените compute_sentence_proba на вашу новую функцию и прогоните получившийся алгоритм на данных
5) сравните предсказания с предсказанием изначального correct_text_with_lm, проверьте метрики и посмотрите на различие в ошибках и исправлениях, найдите несколько примеров отличий в предсказаниях этих подходов

In [13]:
%pip install textdistance


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
import re
import numpy as np
import textdistance
from string import punctuation
from collections import Counter
from pymystem3 import Mystem
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [24]:
morph_analyzer = Mystem(grammar_info=True, disambiguation=False, entire_input=False)

In [17]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [18]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [133]:
def grammar_info(sent):
    grs = morph_analyzer.analyze(sent)
    sent = [x['analysis'][0]['gr'] for x in grs if x['analysis']]
    return sent

def sentensize(text):
    return sent_tokenize(text)

def normalize(text):
    normalized_text = [re.sub("[«»]+", '', word.strip(punctuation)) for word in text.split()]
    normalized_text = [word.lower() for word in normalized_text if word]
    return normalized_text

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def get_statistics(sentences):
    unigrams, bigrams = Counter(), Counter()
    for sent in sentences:
        for i in range(len(sent)):
            unigrams[sent[i]] += 1
            if i>=1:
                bigrams[(sent[i-1], sent[i])] += 1
    return unigrams, bigrams


def analyze_data(corpus, gr_feats=False):
    sentences = sentensize(corpus)
    if gr_feats:
        sentences_w_gram_feats = [grammar_info(sent) for sent in sentences]
        unigrams, bigrams = get_statistics(sentences_w_gram_feats)
    else:
        tokenized_sentences = [normalize(sent) for sent in sentences]
        unigrams, bigrams = get_statistics(tokenized_sentences)
    return unigrams, bigrams

In [ ]:
# основной корпус
corpus = open('wiki_data.txt', encoding='utf8').read()
# словарь
vocab = Counter(re.findall(r'\w+', corpus.lower()))
# размер словаря
N = sum(vocab.values())
id2word = {i:word for i, word in enumerate(vocab.keys())}

In [28]:
# статистика по грам фичам
unigrams, bigrams = analyze_data(corpus, gr_feats=True)

In [29]:
unigrams['S,жен,неод=(пр,ед|вин,мн|дат,ед|род,ед|им,мн)']

105289

In [30]:
# статистика по обычным словам
regular_unigrams, regular_bigrams = analyze_data(corpus)

In [31]:
# функция вероятности для слова
def P(word, N=N):
    return vocab[word] / N

In [33]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

In [33]:
def compute_sentence_proba_gram(text, unigram_counts, bigram_counts, smoothing=2e-5):
    tags = ["<START>"] + grammar_info(text) + ["<END>"]
    prob = 0.0
    for i in range(1, len(tags)):
        t1, t2 = tags[i - 1], tags[i]
        bigram = (t1, t2)

        if t1 in unigram_counts and bigram in bigram_counts:
            p = bigram_counts[bigram] / unigram_counts[t1]
        else:
            p = smoothing
        prob += np.log(p)
    return prob

In [273]:
def calculate_metrics(true, actual, predicted):
    correct = 0
    total = 0
    total_mistaken = 0
    mistaken_fixed = 0
    total_correct = 0
    correct_broken = 0

    for i in range(len(true)):
        t, a, p = true[i], actual[i], predicted[i]
        if t == p:
            correct += 1
        total += 1
        
        if t == a:
            total_correct += 1
            if t != p:
                correct_broken += 1
        else:
            total_mistaken += 1
            if t == p:
                mistaken_fixed += 1

    return {"total_accuracy": correct/total,
            "fixed_mistakes": mistaken_fixed/total_mistaken,
            "broken_correct_words": correct_broken/total_correct}

In [35]:
def compute_sentence_proba(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return prob

In [36]:
vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [37]:
import itertools

def correct_text_with_lm(text, use_gram=True):
    tokens = normalize(text)
    correction_sets = [] 
    
    for word in tokens:
        if predict_mistaken(word, vocab):
            preds = get_closest_hybrid_match(word, X, vec)
            preds = [p[0] for p in preds]
            correction_sets.append(preds)
        else:
            correction_sets.append([word])

    possible_sentences = [" ".join(words) 
                          for words in itertools.product(*correction_sets)]

    if use_gram:
        scorer = lambda s: compute_sentence_proba_gram(s, unigrams, bigrams)
    else:
        scorer = lambda s: compute_sentence_proba(s, regular_unigrams, regular_bigrams)

    best_sentence = max(possible_sentences, key=scorer)
    return best_sentence

In [38]:
y_true = []
y_actual = []
y_preds = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    corrected = correct_text_with_lm(bad[i]).split()
    
    for i in range(len(word_pairs)):
        true_word = word_pairs[i][0]
        actual_word = word_pairs[i][1]
        pred = corrected[i]
            
        y_preds.append(pred)
        y_true.append(true_word)
        y_actual.append(actual_word)

100%|██████████| 915/915 [08:59<00:00,  1.70it/s]


In [39]:
metrics = calculate_metrics(y_true, y_actual, y_preds)
metrics

{'total_accuracy': 0.8364182091045522,
 'fixed_mistakes': 0.3392857142857143,
 'broken_correct_words': 0.09004249454461927}

In [40]:
y_true_reg = []
y_actual_reg = []
y_preds_reg = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    corrected = correct_text_with_lm(bad[i], use_gram=False).split()
    
    for i in range(len(word_pairs)):
        true_word = word_pairs[i][0]
        actual_word = word_pairs[i][1]
        pred = corrected[i]
            
        y_preds_reg.append(pred)
        y_true_reg.append(true_word)
        y_actual_reg.append(actual_word)

100%|██████████| 915/915 [07:41<00:00,  1.98it/s]


In [43]:
regular_metrics = calculate_metrics(y_true_reg, y_actual_reg, y_preds_reg)
regular_metrics

{'total_accuracy': 0.8467233616808404,
 'fixed_mistakes': 0.4192546583850932,
 'broken_correct_words': 0.09004249454461927}

In [44]:
print('Слова, где обе ошиблись, но по-разному')
i = 0
for true_pred, gram_pred, actual_pred, reg_pred in zip(y_true, y_preds, y_actual, y_preds_reg):
    if i == 15:
        break
    if true_pred != gram_pred and true_pred != reg_pred and reg_pred != gram_pred:
        i += 1
        s = ''
        print("ТИП ОШИБКИ:", end="\t")
        if actual_pred == true_pred:
            s = 'FALSE POSITIVE\t(слово не нуждалось в исправлении)'
        else:
            s = 'НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ'
        print(s, end='\n\n')
        print(f"Реальное слово: {actual_pred}\t\\\tКорректное исправление: {true_pred:}", end='\n\n')
        print("Предсказания:")
        print(f"\t - Стандартная функция: {reg_pred}")
        print(f"\t - Функция на грамматических признаках: {gram_pred}", end='\n\n')

Слова, где обе ошиблись, но по-разному
ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: симпатичнейшое	\	Корректное исправление: симпатичнейшее

Предсказания:
	 - Стандартная функция: симпатичнее
	 - Функция на грамматических признаках: симпатичной

ТИП ОШИБКИ:	FALSE POSITIVE	(слово не нуждалось в исправлении)

Реальное слово: шпионское	\	Корректное исправление: шпионское

Предсказания:
	 - Стандартная функция: шпионско
	 - Функция на грамматических признаках: шпионской

ТИП ОШИБКИ:	FALSE POSITIVE	(слово не нуждалось в исправлении)

Реальное слово: гламурный	\	Корректное исправление: гламурный

Предсказания:
	 - Стандартная функция: гламурным
	 - Функция на грамматических признаках: гламурные

ТИП ОШИБКИ:	FALSE POSITIVE	(слово не нуждалось в исправлении)

Реальное слово: услышанная	\	Корректное исправление: услышанная

Предсказания:
	 - Стандартная функция: услышанным
	 - Функция на грамматических признаках: услышанные

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: пояним	

Из интересного: модели пытаются корректные слова исправить, когда они в каком-то падеже (не в номинативе), глаголы в менее, видимо, частотных лице-численных формах, и в принципе редкие солова. СМ:

```Реальное слово: билетным	\	Корректное исправление: билетным

Предсказания:
	 - Стандартная функция: билетные
	 - Функция на грамматических признаках: билетных

Реальное слово: капулетти	\	Корректное исправление: капулетти

Предсказания:
	 - Стандартная функция: катапультируется
	 - Функция на грамматических признаках: паулетты



In [45]:
print('Слова, где ошиблась грамматическая')
i = 0
for true_pred, gram_pred, actual_pred, reg_pred in zip(y_true, y_preds, y_actual, y_preds_reg):
    if i == 15:
        break
    if true_pred != gram_pred and true_pred == reg_pred:
        i += 1
        s = ''
        print("ТИП ОШИБКИ:", end="\t")
        if actual_pred == true_pred:
            s = 'FALSE POSITIVE\t(слово не нуждалось в исправлении)'
        else:
            s = 'НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ'
        print(s, end='\n\n')
        print(f"Реальное слово: {actual_pred}\t\\\tКорректное исправление: {true_pred:}", end='\n\n')
        print("Предсказания:")
        print(f"\t - Стандартная функция: {reg_pred}")
        print(f"\t - Функция на грамматических признаках: {gram_pred}", end='\n\n')

Слова, где ошиблась грамматическая
ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: предлагю	\	Корректное исправление: предлагаю

Предсказания:
	 - Стандартная функция: предлагаю
	 - Функция на грамматических признаках: предлагают

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: выходгых	\	Корректное исправление: выходных

Предсказания:
	 - Стандартная функция: выходных
	 - Функция на грамматических признаках: выходы

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: компютерная	\	Корректное исправление: компьютерная

Предсказания:
	 - Стандартная функция: компьютерная
	 - Функция на грамматических признаках: суперкомпьютерная

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: отвественность	\	Корректное исправление: ответственность

Предсказания:
	 - Стандартная функция: ответственность
	 - Функция на грамматических признаках: ответственностью

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: коментариев	\	Корректное исправление: комментариев

Предсказания:
	 -

In [46]:
print('Слова, где ошиблась стандартная')
i = 0
for true_pred, gram_pred, actual_pred, reg_pred in zip(y_true, y_preds, y_actual, y_preds_reg):
    if i == 15:
        break
    if true_pred == gram_pred and true_pred != reg_pred:
        i += 1
        s = ''
        print("ТИП ОШИБКИ:", end="\t")
        if actual_pred == true_pred:
            s = 'FALSE POSITIVE\t(слово не нуждалось в исправлении)'
        else:
            s = 'НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ'
        print(s, end='\n\n')
        print(f"Реальное слово: {actual_pred}\t\\\tКорректное исправление: {true_pred:}", end='\n\n')
        print("Предсказания:")
        print(f"\t - Стандартная функция: {reg_pred}")
        print(f"\t - Функция на грамматических признаках: {gram_pred}", end='\n\n')

Слова, где ошиблась стандартная
ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: баръер	\	Корректное исправление: барьер

Предсказания:
	 - Стандартная функция: барбер
	 - Функция на грамматических признаках: барьер

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: ооочень	\	Корректное исправление: очень

Предсказания:
	 - Стандартная функция: сорочень
	 - Функция на грамматических признаках: очень

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: оооочень	\	Корректное исправление: очень

Предсказания:
	 - Стандартная функция: сорочень
	 - Функция на грамматических признаках: очень

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: получаеться	\	Корректное исправление: получается

Предсказания:
	 - Стандартная функция: получаться
	 - Функция на грамматических признаках: получается

ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: движениие	\	Корректное исправление: движение

Предсказания:
	 - Стандартная функция: движении
	 - Функция на грамматических признаках:

In [95]:
print('Примеры слов, где произошла "гиперкоррекция" у обеих моделей (одинаковая)')
hyper_reg = []
hyper_gram = []
i = 0
for true_pred, gram_pred, actual_pred, reg_pred in zip(y_true, y_preds, y_actual, y_preds_reg):
    if i == 15:
        break
    if true_pred != gram_pred and true_pred != reg_pred:
        s = ''
        if actual_pred == true_pred:
            i += 1
            hyper_reg.append((true_pred, reg_pred))
            hyper_gram.append((true_pred, gram_pred))

intersect = set(hyper_reg).intersection(set(hyper_gram))
print(f"Пересечение: {len(intersect)}")
for word in intersect:
    print('Слово: {:>30}\t'.format(word[0]), end='')
    print('{:^10}'.format('Исправление'), end="")
    print('{:>20}'.format(word[1]))

Примеры слов, где произошла "гиперкоррекция" у обеих моделей (одинаковая)
Пересечение: 9
Слово:                       люминала	Исправление            номинала
Слово:           дорожно-транспортных	Исправление    автотранспортных
Слово:                         снятся	Исправление           выяснятся
Слово:                       почте.ру	Исправление               почте
Слово:                     superheadz	Исправление       superspeedway
Слово:                         сетуют	Исправление            советуют
Слово:                         язычки	Исправление            язычники
Слово:          аварийно-спасательных	Исправление   горноспасательных
Слово:                           clap	Исправление             clapier


## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (обратите внимание, что для одного удаления может быть несколько правильных слов!) 
3) При исправлении слова с опечаткой сначала само слово проверятся по словарю удаления, а затем для этого слова генерируются все варианты удалений, и каждый вариант проверяется по словарю удалений. Если в словаре удалений таким образом находится совпадение, то соответствующее ему правильное слово становится исправлением.
Если совпадений несколько, то выбирается наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [4]:
import re
from collections import defaultdict, Counter

In [226]:
def generate_deletes_step(word, distance, max_distance, deletes):
    if distance > max_distance:
        return

    for i in range(len(word)):
        deleted = word[:i] + word[i+1:]
        if deleted not in deletes:
            deletes.add(deleted)
            generate_deletes_step(deleted, distance + 1, max_distance, deletes)


def generate_deletes(word, max_distance=MAX_EDIT_DISTANCE):
    deletes = set()
    generate_deletes_step(word, 1, max_distance, deletes)
    return deletes

In [227]:
def build_dictionary(corpus, max_distance=MAX_EDIT_DISTANCE):
    words = normalize(corpus)
    for w in words:
        w = w.lower()
        dictionary[w] += 1

    for w in dictionary:
        for d in generate_deletes(w, max_distance):
            deletion_index[d].append(w)

In [228]:
def symspell_candidates(input_word, dictionary, deletion_index, max_distance=MAX_EDIT_DISTANCE):
    candidates = set()

    if input_word in dictionary:
        candidates.add(input_word)
        return candidates 

    if input_word in deletion_index:
        candidates.update(deletion_index[input_word])

    for d in generate_deletes(input_word, max_distance):
        if d in deletion_index:
            candidates.update(deletion_index[d])

    return candidates

In [229]:
def rank_candidates(input_word, candidates, dictionary):
    scored = []
    for cand in candidates:
        d = textdistance.levenshtein(input_word, cand)
        freq = dictionary[cand]
        scored.append((cand, d, freq))
    
    # сортировка по расстоянию, потом по частоте в словаре
    scored.sort(key=lambda x: (x[1], -x[2], x[0]))
    return scored

In [230]:
def symspell_best(input_word, candidates, dictionary):
    ranked = rank_candidates(input_word, candidates, dictionary)
    return ranked[0][0] if ranked else None

In [231]:
def correct_text_with_symspell(text, vocab, deletion_index, max_distance=2):
    tokens = normalize(text)
    corrected_tokens = []

    for word in tokens:
        if predict_mistaken(word, vocab):
            candidates = symspell_candidates(word, vocab, deletion_index, max_distance=max_distance)
            correction = symspell_best(word, candidates, vocab)
            corrected_tokens.append(correction if correction else word)
        else:
            corrected_tokens.append(word)

    return " ".join(corrected_tokens)

In [232]:
corpus = open('wiki_data.txt', encoding='utf8').read()

# словарь
dictionary = Counter()
# словарь удалений
deletion_index = defaultdict(list)

build_dictionary(corpus)

In [233]:
DELETION_INDEX = deletion_index
DICTIONARY = dictionary

In [236]:
y_true = []
y_actual = []
y_preds = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])

    corrected = correct_text_with_symspell(bad[i], dictionary, deletion_index).split()

    for j in range(len(word_pairs)):
        true_word = word_pairs[j][0]
        actual_word = word_pairs[j][1]
        pred = corrected[j]

        y_true.append(true_word)
        y_actual.append(actual_word)
        y_preds.append(pred)

100%|██████████| 915/915 [00:00<00:00, 1429.80it/s]


In [237]:
metrics = calculate_metrics(y_true, y_actual, y_preds)
metrics

{'total_accuracy': 0.870935467733867,
 'fixed_mistakes': 0.5170807453416149,
 'broken_correct_words': 0.07671988055587459}

In [238]:
print('Слова, где ошиблась модель')
i = 0
for true_pred, model_pred, actual_pred in zip(y_true, y_preds, y_actual):
    if i == 15:
        break
    if true_pred != model_pred:
        i += 1
        s = ''
        print("ТИП ОШИБКИ:", end="\t")
        if actual_pred == true_pred:
            s = 'FALSE POSITIVE\t(слово не нуждалось в исправлении)'
        else:
            s = 'НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ'
        print(s, end='\n\n')
        print(f"Реальное слово: {actual_pred}\t\\\tКорректное исправление: {true_pred:}", end='\n\n')
        print(f"Предсказание: {model_pred}")

Слова, где ошиблась модель
ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: симпатичнейшое	\	Корректное исправление: симпатичнейшее

Предсказание: симпатичнейшое
ТИП ОШИБКИ:	FALSE POSITIVE	(слово не нуждалось в исправлении)

Реальное слово: шпионское	\	Корректное исправление: шпионское

Предсказание: шпионской
ТИП ОШИБКИ:	FALSE POSITIVE	(слово не нуждалось в исправлении)

Реальное слово: гламурный	\	Корректное исправление: гламурный

Предсказание: гламурным
ТИП ОШИБКИ:	FALSE POSITIVE	(слово не нуждалось в исправлении)

Реальное слово: clap	\	Корректное исправление: clap

Предсказание: cap
ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: опофеозом	\	Корректное исправление: апофеозом

Предсказание: опофеозом
ТИП ОШИБКИ:	FALSE POSITIVE	(слово не нуждалось в исправлении)

Реальное слово: услышанная	\	Корректное исправление: услышанная

Предсказание: услышанное
ТИП ОШИБКИ:	НЕПРАВИЛЬНОЕ ИСПРАВЛЕНИЕ

Реальное слово: пояним	\	Корректное исправление: поясним

Предсказание: одним
ТИП О

# Задание 3 (2 балла)

Используя любой из алгоритмов из семинара или домашки, детально проанализируйте получаемые ошибки. Улучшите алгоритм так, чтобы исправить ошибки. Улучшения в алгоритме должны быть общими, не привязанными к конкретным словам (например, словарь исключений не будет считаться). За каждое улучшение, которое исправляет 5+ ошибок вы получите 0.5 балла (максимум 2 в целом)

In [239]:
for true_pred, model_pred, actual_pred in zip(y_true, y_preds, y_actual):
    if true_pred != model_pred:
        print(f"Реальное слово: {actual_pred}\t\\\tКорректное исправление: {true_pred:}")
        print(f"Предсказание: {model_pred}", end='\n\n')

Реальное слово: симпатичнейшое	\	Корректное исправление: симпатичнейшее
Предсказание: симпатичнейшое

Реальное слово: шпионское	\	Корректное исправление: шпионское
Предсказание: шпионской

Реальное слово: гламурный	\	Корректное исправление: гламурный
Предсказание: гламурным

Реальное слово: clap	\	Корректное исправление: clap
Предсказание: cap

Реальное слово: опофеозом	\	Корректное исправление: апофеозом
Предсказание: опофеозом

Реальное слово: услышанная	\	Корректное исправление: услышанная
Предсказание: услышанное

Реальное слово: пояним	\	Корректное исправление: поясним
Предсказание: одним

Реальное слово: язычки	\	Корректное исправление: язычки
Предсказание: языки

Реальное слово: оччччень	\	Корректное исправление: очень
Предсказание: оччччень

Реальное слово: нащщот	\	Корректное исправление: насчет
Предсказание: нагота

Реальное слово: сетуют	\	Корректное исправление: сетуют
Предсказание: сетует

Реальное слово: основая	\	Корректное исправление: основная
Предсказание: основан

Ре

Сначала для симспела попробуем добавить фильтрацию по морфологии, так как часть ошибок завязано чисто на ней:

```Реальное слово: пояним	\	Корректное исправление: поясним
Предсказание: одним

=> часть речи

```Реальное слово: дорожно-транспортных	\	Корректное исправление: дорожно-транспортных
Предсказание: дорожно-транспортные

=> падеж

Реальное слово: иностранка	\	Корректное исправление: иностранка
Предсказание: иностранца
=> род



In [47]:
import math
from functools import lru_cache
import pymorphy3

In [48]:
morph = pymorphy3.MorphAnalyzer()

In [ ]:
def ending_match(word1, word2, length=3):
    return int(word1[-length:] == word2[-length:])


def grammar_match(word1, word2, ending=False):
    parse1 = morph.parse(word1)[0]
    parse2 = morph.parse(word2)[0]

    cumm = 0
    
    if parse1.tag.POS == parse2.tag.POS:
        cumm += 1

    gender_match = (parse1.tag.gender == parse2.tag.gender)
    if gender_match:
        cumm += 1

    number_match = (parse1.tag.number == parse2.tag.number)
    if number_match:
        cumm += 1
    
    case_match = (parse1.tag.case == parse2.tag.case)
    if case_match: 
        cumm += 1 

    if ending:
        cumm += ending_match(word1, word2)
    return cumm


def candidate_score(input_word, candidate, dictionary, weight=None, ending=False):
    freq = dictionary.get(candidate, 0)
    gm = grammar_match(input_word, candidate, ending=ending)
    if weight is None:
        dist =  textdistance.damerau_levenshtein(input_word, candidate)
    else:
        dist = weighted_damerau_levenshtein(input_word, candidate, costs=weight)

    score = (candidate, dist, freq, gm)
    return score

In [392]:
def correct_word(input_word, dictionary, deletion_index, max_distance=2, ending=False, weight=None):
    candidates = set()
    
    if input_word in dictionary:
        candidates.add(input_word)
    
    deletes = generate_deletes(input_word, max_distance=max_distance)
    for d in deletes:
        if d in deletion_index:
            candidates.update(deletion_index[d])
    
    if not candidates:
        return input_word
    
    scored = [(candidate_score(input_word, c, dictionary, weight=weight)) for c in candidates]
    scored.sort(key=lambda x: (x[1], -x[2], -x[3], x[0]))
    return scored[0][0]

In [393]:
def correct_text_with_symspell_morph(text, vocab, deletion_index, max_distance=2, ending=False, weight=None):
    tokens = normalize(text)
    corrected_tokens = []

    for word in tokens:
        if predict_mistaken(word, vocab):
            candidate = correct_word(word, vocab, deletion_index, max_distance=max_distance, ending=ending, weight=weight)
            corrected_tokens.append(candidate if candidate else word)
        else:
            corrected_tokens.append(word)

    return " ".join(corrected_tokens)

In [270]:
y_true_new = []
y_actual_new = []
y_preds_new = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])

    corrected = correct_text_with_symspell_morph(bad[i], dictionary, deletion_index, max_distance=2).split()

    for j in range(len(word_pairs)):
        true_word = word_pairs[j][0]
        actual_word = word_pairs[j][1]
        pred = corrected[j]

        y_true_new.append(true_word)
        y_actual_new.append(actual_word)
        y_preds_new.append(pred)

100%|██████████| 915/915 [00:28<00:00, 31.89it/s]


In [271]:
metrics = calculate_metrics(y_true_new, y_actual_new, y_preds_new)
metrics

{'total_accuracy': 0.871735867933967,
 'fixed_mistakes': 0.5232919254658385,
 'broken_correct_words': 0.07671988055587459}

С помощью добавления грамматики и замены функции с обычного Левенштейна, на доработку с транспозицией (Дамерау-Левенштейн) получилось чуток поднять качество (на 12 пунктов...):

In [277]:
print(f"Правильных разборов у обычной: {0.870935467733867*len(y_true_new)}")
print(f"Правильных разборов у улучшенной: {0.871735867933967*len(y_true_new)}")


Правильных разборов у обычной: 8705.0
Правильных разборов у улучшенной: 8713.0


In [272]:
for true_pred, model_pred, actual_pred in zip(y_true_new, y_preds_new, y_actual_new):
    if true_pred != model_pred:
        print(f"Реальное слово: {actual_pred}\t\\\tКорректное исправление: {true_pred:}")
        print(f"Предсказание: {model_pred}", end='\n\n')

Реальное слово: симпатичнейшое	\	Корректное исправление: симпатичнейшее
Предсказание: симпатичнейшое

Реальное слово: шпионское	\	Корректное исправление: шпионское
Предсказание: шпионской

Реальное слово: гламурный	\	Корректное исправление: гламурный
Предсказание: гламурным

Реальное слово: clap	\	Корректное исправление: clap
Предсказание: cap

Реальное слово: опофеозом	\	Корректное исправление: апофеозом
Предсказание: опофеозом

Реальное слово: услышанная	\	Корректное исправление: услышанная
Предсказание: услышанное

Реальное слово: пояним	\	Корректное исправление: поясним
Предсказание: одним

Реальное слово: язычки	\	Корректное исправление: язычки
Предсказание: языки

Реальное слово: оччччень	\	Корректное исправление: очень
Предсказание: оччччень

Реальное слово: нащщот	\	Корректное исправление: насчет
Предсказание: нагота

Реальное слово: сетуют	\	Корректное исправление: сетуют
Предсказание: сетует

Реальное слово: основая	\	Корректное исправление: основная
Предсказание: основан

Ре

Еще я хотела попробовать использовать взвешенную метрику Левенштейна с параметрами опечаток на клавиатуре

In [367]:
transpose_costs = np.ones((32, 32))

In [384]:
# я хотела использовать библиотеку библиотеку но там для кириллических символов не подходит
# поэтому DP функция украденная из интернета и чуток доработанная

def sub_cost_matrix(a, b, costs):
    idx_a = ord(a) - 1072
    idx_b = ord(b) - 1072

    if 0 <= idx_a < costs.shape[0] and 0 <= idx_b < costs.shape[1]:
        return costs[idx_a, idx_b]
    else:
        return 2.0


import numpy as np

def weighted_damerau_levenshtein(s1, s2, costs):
    n, m = len(s1), len(s2)
    dp = np.zeros((n+1, m+1), dtype=float)

    for i in range(n+1):
        dp[i][0] = i
    for j in range(m+1):
        dp[0][j] = j

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = sub_cost_matrix(s1[i-1], s2[j-1], costs)
            dp[i][j] = min(
                dp[i-1][j] + 1,        # удаление
                dp[i][j-1] + 1,        # вставка
                dp[i-1][j-1] + cost    # замена
            )
            # транспозиция
            if i>1 and j>1 and s1[i-1]==s2[j-2] and s1[i-2]==s2[j-1]:
                dp[i][j] = min(dp[i][j], dp[i-2][j-2] + 0.5)

    return dp[n][m]

In [375]:
keyboard_pos = {
    'й': (0, 0), 'ц': (0, 1), 'у': (0, 2), 'к': (0, 3), 'е': (0, 4),
    'н': (0, 5), 'г': (0, 6), 'ш': (0, 7), 'щ': (0, 8), 'з': (0, 9), 'х': (0, 10),

    'ф': (1, 0), 'ы': (1, 1), 'в': (1, 2), 'а': (1, 3), 'п': (1, 4),
    'р': (1, 5), 'о': (1, 6), 'л': (1, 7), 'д': (1, 8), 'ж': (1, 9), 'э': (1, 10),

    'я': (2, 1), 'ч': (2, 2), 'с': (2, 3), 'м': (2, 4), 'и': (2, 5),
    'т': (2, 6), 'ь': (2, 7), 'б': (2, 8), 'ю': (2, 9)
}

In [376]:
import math

def key_distance(a, b, keyboard_pos):
    if a not in keyboard_pos or b not in keyboard_pos:
        return 1.0
    (r1, c1) = keyboard_pos[a]
    (r2, c2) = keyboard_pos[b]
    # евклидова метрика
    return math.sqrt((r1 - r2)**2 + (c1 - c2)**2)

In [377]:
costs = np.ones((32, 32), dtype=np.float64)
for i in range(1072, 1104):
    for j in range(1072, 1104):
        dist = key_distance(chr(i), chr(j), keyboard_pos=keyboard_pos)
        costs[abs(1072-i), abs(1072-j)] = dist

In [400]:
y_true_end = []
y_actual_end = []
y_preds_end = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])

    corrected = correct_text_with_symspell_morph(bad[i], dictionary, deletion_index, max_distance=3, weight=costs).split()

    for j in range(len(word_pairs)):
        true_word = word_pairs[j][0]
        actual_word = word_pairs[j][1]
        pred = corrected[j]

        y_true_end.append(true_word)
        y_actual_end.append(actual_word)
        y_preds_end.append(pred)

100%|██████████| 915/915 [02:17<00:00,  6.64it/s]


In [399]:
metrics = calculate_metrics(y_true_end, y_actual_end, y_preds_end)
metrics

{'total_accuracy': 0.8663331665832916,
 'fixed_mistakes': 0.3579192546583851,
 'broken_correct_words': 0.058458711381646954}

Качество ухудшилось... можно попробовать фонетическую модель, что в принципе по данным похоже, что ошибки именно из-за того, что люди пишут так же, как и слышат...

После поисков по этой тематике я наткнулась на работу Сорокина и Шавриной 2016 (https://www.researchgate.net/publication/303813582_Automatic_spelling_correction_for_Russian_social_media_texts), поэтому данные по фонетическим классам взяла оттуда

In [410]:
phonetic_classes = {
    1:  "а о ы у я",      
    3:  "и е ё ю я э",    
    5:  "б п",
    6:  "в ф",
    7:  "д т",
    8:  "г к х",
    9:  "л",
    10: "р",
    11: "м",
    12: "н",
    13: "з с",
    14: "й",
    15: "щ ч",
    16: "ж ш",
    17: "ц"
}

groups = {
    "vowels": {1,3},
    "labials": {5,6},
    "dentals": {7,13,17},
    "palatals": {14,15},
    "velars": {8},
    "liquids": {9,10},
    "nasals": {11,12},
    "shush": {16},
}

In [411]:
letter_to_classes = {}

for cls, letters in phonetic_classes.items():
    for letter in letters.split():
        letter_to_classes.setdefault(letter, set()).add(cls)

In [412]:
def phonetic_sub_cost(a, b):
    if a == b:
        return 0.0
    
    cls_a = letter_to_classes.get(a, set())
    cls_b = letter_to_classes.get(b, set())
    
    if not cls_a or not cls_b:
        return 1.0
    
    if cls_a & cls_b:
        return 0.2
    
    for grp, members in groups.items():
        if cls_a.issubset(members) and cls_b.issubset(members):
            return 0.7
    
    return 1.0


In [ ]:
alphabet = [chr(i) for i in range(1072, 1104)]

sub_costs = np.zeros((32, 32), dtype=np.float64)

for i, c1 in enumerate(alphabet):
    for j, c2 in enumerate(alphabet):
        sub_costs[i, j] = phonetic_sub_cost(c1, c2)

In [415]:
y_true_phone = []
y_actual_phone = []
y_preds_phone = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])

    corrected = correct_text_with_symspell_morph(bad[i], dictionary, deletion_index, max_distance=3, weight=sub_costs).split()

    for j in range(len(word_pairs)):
        true_word = word_pairs[j][0]
        actual_word = word_pairs[j][1]
        pred = corrected[j]

        y_true_phone.append(true_word)
        y_actual_phone.append(actual_word)
        y_preds_phone.append(pred)

100%|██████████| 915/915 [02:13<00:00,  6.85it/s]


In [416]:
metrics = calculate_metrics(y_true_end, y_actual_phone, y_preds_phone)
metrics

{'total_accuracy': 0.8649324662331166,
 'fixed_mistakes': 0.5015527950310559,
 'broken_correct_words': 0.08131388537957965}

In [417]:
# если комбинировать...
def candidate_score(input_word, candidate, dictionary, weight=None, ending=False):
    freq = dictionary.get(candidate, 0)
    gm = grammar_match(input_word, candidate, ending=ending)
    dist =  textdistance.damerau_levenshtein(input_word, candidate)
    phone_dist = weighted_damerau_levenshtein(input_word, candidate, costs=weight)

    score = (candidate, dist+phone_dist/2, freq, gm)
    return score

In [418]:
y_true_phone = []
y_actual_phone = []
y_preds_phone = []

for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])

    corrected = correct_text_with_symspell_morph(bad[i], dictionary, deletion_index, max_distance=3, weight=sub_costs).split()

    for j in range(len(word_pairs)):
        true_word = word_pairs[j][0]
        actual_word = word_pairs[j][1]
        pred = corrected[j]

        y_true_phone.append(true_word)
        y_actual_phone.append(actual_word)
        y_preds_phone.append(pred)

100%|██████████| 915/915 [02:20<00:00,  6.54it/s]


In [419]:
metrics = calculate_metrics(y_true_end, y_actual_phone, y_preds_phone)
metrics

{'total_accuracy': 0.8701350675337669,
 'fixed_mistakes': 0.5419254658385093,
 'broken_correct_words': 0.08131388537957965}

Я пробовала разные комбинации, но результаты дало только небольшое добавление морфологии + добавлению в дистанцию транспозиции классической. Причем, когда я делала полноценную фильтрацию, где отбрасывала кандидатов, нарушающие разные признаки оригинального слова, – результаты становились даже хуже, чем при самом обычно SymSpell. Итого у улучшенной результат лучше только на 12 пунктов, but it was fun.